Lung Cancer Research Project



Clean Up Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
import requests
import hvplot.pandas

# Import the OpenWeatherMap API key
from api_key import weather_api_key


In [ ]:
#set file path
cancer_data_path = "../Resources/cancer patient data sets.csv"
#read the file path
cancer_data = pd.read_csv(cancer_data_path)
cancer_data.head()

In [ ]:
#cleaning data - checking that all columns are filled out
cancer_data.info()

In [ ]:
#cleaning data - create new data frame that removes excess columnns
clean_cancer = cancer_data[["Patient Id", "Age", "Gender", "Air Pollution", "Alcohol use", "Genetic Risk", "Obesity", "Smoking", "Passive Smoker", "Level"]]
clean_cancer.head()

Question 1 - How do the risk factors differ for different age groups of patients?

Question 2 - How do the risk factors differ for male vs female patients?

Question 3 - What is the relation of pollution to the level of cancer in patients?

Question 4 - What is the relation of smoking habits to the level of cancer in patients?

Conclusion